In [1]:
import os
import pickle
import torch
import seaborn as sns
import tempfile
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy

from molecule_featurizer import MoleculeFeaturizer
from rdkit import Chem
from ccdc.docking import Docker
from ccdc.io import MoleculeReader, EntryReader
from ccdc_rdkit_connector import CcdcRdkitConnector
from ccdc.descriptors import MolecularDescriptors
from ccdc.io import Entry
from torch_geometric.data import Batch
from litschnet import LitSchNet
from mol_viewer import MolViewer
from multiprocessing import Pool

In [2]:
data_dir = 'data/'
pdbbind_refined_dir = '/home/benoit/PDBBind/PDBbind_v2020_refined/refined-set/'
pdbbind_general_dir = '/home/benoit/PDBBind/PDBbind_v2020_other_PL/v2020-other-PL/'

In [3]:
with open(os.path.join(data_dir, 'random_splits', f'test_smiles_random_split_0.txt'), 'r') as f :
    test_smiles = f.readlines()
    test_smiles = [smiles.strip() for smiles in test_smiles]
    
with open(os.path.join(data_dir, 'raw', 'ccdc_generated_conf_ensemble_library.p'), 'rb') as f :
    cel = pickle.load(f)

In [4]:
connector = CcdcRdkitConnector()

encoder_path = os.path.join(data_dir, 'molecule_encoders.p')
if os.path.exists(encoder_path) : # Load existing encoder
    with open(encoder_path, 'rb') as f:
        mol_encoders = pickle.load(f)
mol_featurizer = MoleculeFeaturizer(mol_encoders)

experiment_name = f'random_split_0_new'
if experiment_name in os.listdir('lightning_logs') :
    checkpoint_name = os.listdir(os.path.join('lightning_logs', experiment_name, 'checkpoints'))[0]
    checkpoint_path = os.path.join('lightning_logs', experiment_name, 'checkpoints', checkpoint_name)
    litschnet = LitSchNet.load_from_checkpoint(checkpoint_path=checkpoint_path)
litschnet.eval()

LitSchNet(
  (schnet): SchNet(hidden_channels=128, num_filters=128, num_interactions=6, num_gaussians=50, cutoff=10.0)
  (leaky_relu): LeakyReLU(negative_slope=0.01)
  (sigmoid): Sigmoid()
)

In [5]:
def gold_docking(native_ligand,
                 ligand,
                 protein_file: str,
                 reference_ligand_file: str,
                 dock_id: str,
                 n_pose_per_conf: int=100) :
    
    connector = CcdcRdkitConnector()
    
    docker = Docker()
    settings = docker.settings
    
    settings.add_protein_file(protein_file)
    settings.reference_ligand_file = reference_ligand_file
    
    # Define binding site
    protein = settings.proteins[0]
    settings.binding_site = settings.BindingSiteFromLigand(protein, native_ligand, 8.0)
        
    ligand_prep = Docker.LigandPreparation()
        
    output_dir = os.path.abspath(os.path.join('gold_docking_postprocessing', f'{dock_id}'))
    if not os.path.exists(output_dir) :
        os.mkdir(output_dir)
    settings.output_directory = output_dir
    settings.output_file = os.path.join(output_dir, f'docked_ligands_{dock_id}.mol2')
     
    ligand_file = os.path.join(output_dir, f'{dock_id}.mol2')
    ligand = ligand_prep.prepare(Entry.from_molecule(ligand))
    mol2_string = ligand.to_string(format='mol2')
    with open(ligand_file, 'w') as writer :
        writer.write(mol2_string)
    settings.add_ligand_file(ligand_file, n_pose_per_conf)

    settings.fitness_function = 'plp'
    settings.autoscale = 10.
    settings.early_termination = False
    
    start_time = time.time()
    results = docker.dock(file_name=os.path.join(output_dir, f'api_gold.conf'))
    print(results.return_code)
    runtime = time.time() - start_time
    
    results = Docker.Results(settings)
    ligands = results.ligands
    
    return ligands, runtime, ligand.molecule

In [6]:
def docking_performance(ligand_subset, native_ligand) :
    results_d = {}
    
    ligand_scores = [l.fitness() for l in ligand_subset]
    ligand_scores = np.array(ligand_scores)
    ligand_mols = [l.molecule for l in ligand_subset]
    for ligand_mol in ligand_mols :
        ligand_mol.remove_atoms([atom for atom in ligand_mol.atoms if atom.atomic_number < 1])
    
    # find top score pose
    top_score_index = ligand_scores.argmax()
    top_score_pose = ligand_mols[top_score_index]

    # find best pose 
    rmsds_to_real_pose = np.array([MolecularDescriptors.rmsd(native_ligand, ligand_mol, overlay=False) for ligand_mol in ligand_mols])
    best_pose_index = rmsds_to_real_pose.argmin()
    best_pose = ligand_mols[best_pose_index]

    top_score = ligand_scores.max()
    min_rmsd = rmsds_to_real_pose.min()

    results_d['top_score_pose'] = connector.ccdc_mol_to_rdkit_mol(top_score_pose)
    results_d['top_score'] = top_score
    results_d['best_pose'] = connector.ccdc_mol_to_rdkit_mol(best_pose)
    results_d['min_rmsd'] = min_rmsd
    min_rmsd_top_pose = MolecularDescriptors.rmsd(native_ligand, top_score_pose, overlay=False)
    results_d['min_rmsd_top_pose'] = min_rmsd_top_pose
    results_d['docking_power'] = min_rmsd_top_pose < 2

    return results_d

In [12]:
def dock_smiles(params) :
    dock_id, smiles, ce = params
    result_d = {}
    result_d['ranking'] = {}
    
    pdb_id = ce.mol.GetConformer(0).GetProp('PDB_ID')

    if pdb_id in os.listdir(pdbbind_refined_dir) :
        protein_file = os.path.join(pdbbind_refined_dir, pdb_id, f'{pdb_id}_protein.pdb')
        ligand_file = os.path.join(pdbbind_refined_dir, pdb_id, f'{pdb_id}_ligand.mol2')
    else :
        protein_file = os.path.join(pdbbind_general_dir, pdb_id, f'{pdb_id}_protein.pdb')
        ligand_file = os.path.join(pdbbind_general_dir, pdb_id, f'{pdb_id}_ligand.mol2')

    rdkit_mol = copy.deepcopy(ce.mol)
    native_ligand = connector.rdkit_conf_to_ccdc_mol(rdkit_mol, conf_id=0)
    
    ligand_prep = Docker.LigandPreparation()
    native_ligand = ligand_prep.prepare(Entry.from_molecule(native_ligand))
    native_ligand = native_ligand.molecule
    
    generated_ids = [conf.GetId() for conf in ce.mol.GetConformers() if conf.HasProp('Generator')]
    gen_conf_id = generated_ids[0]
    ligand = connector.rdkit_conf_to_ccdc_mol(rdkit_mol, conf_id=gen_conf_id)
    
    try :
        ligands, runtime, ligand = gold_docking(native_ligand, ligand, protein_file, ligand_file, dock_id)

        if len(ligands) :
            result_d['runtime'] = runtime

            ligand_scores = [l.fitness() for l in ligands]
            ligand_mols = [l.molecule for l in ligands]

            rdkit_mol.RemoveAllConformers()
            rdkit_mol.AddConformer(ce.mol.GetConformer(0), assignId=True)
            new_confs_ids = connector.ccdc_mols_to_rdkit_mol_conformers(ligand_mols, rdkit_mol)

    #         viewer = MolViewer()
    #         viewer.view(rdkit_mol)

            ids = [conf.GetId() for conf in rdkit_mol.GetConformers()]
            data_list = mol_featurizer.featurize_mol(rdkit_mol)
            data_list = data_list[1:]
            batch = Batch.from_data_list(data_list)

            with torch.no_grad() :
                preds = litschnet(batch).cpu().numpy()
            preds = preds.reshape(-1)

            rmsds = mol_featurizer.get_bioactive_rmsds(rdkit_mol).numpy()[1:]
            energies = [data.energy for data in data_list]

            # ranked by score
            top20_score = np.array(ligand_scores).argsort()[-20:]
            ligand_subset = [l for i, l in enumerate(ligands) if i in top20_score]
            d = docking_performance(ligand_subset, native_ligand)
            result_d['ranking']['score'] = d

            # ranked by model
            top20_preds = preds.argsort()[:20]
            ligand_subset = [l for i, l in enumerate(ligands) if i in top20_preds]
            d = docking_performance(ligand_subset, native_ligand)
            result_d['ranking']['model'] = d

            # ranked by energy
            top20_energy = np.array(energies).argsort()[:20]
            ligand_subset = [l for i, l in enumerate(ligands) if i in top20_energy]
            d = docking_performance(ligand_subset, native_ligand)
            result_d['ranking']['energy'] = d

            with open(os.path.join('gold_docking_postprocessing', str(dock_id), 'results.p'), 'wb') as f :
                pickle.dump(result_d, f)

        else :
            print(f'{smiles} docking was unsucessful')
   
    except Exception as e :
        print(f'Error with smiles {smiles}')
        print(str(e))

In [8]:
# for i, smiles in enumerate(test_smiles) :
#     params = (i, smiles, cel.get_conf_ensemble(smiles))
#     dock_smiles(params)

In [9]:
    #         sns.scatterplot(x=ligand_scores, y=preds[1:])
    #         plt.title('Score vs Pred')
    #         plt.show()

    #         sns.scatterplot(x=real_rmsds, y=preds[1:])
    #         plt.title('RMSD vs Pred')
    #         plt.show()

    #         sns.scatterplot(x=real_rmsds, y=ligand_scores)
    #         plt.title('RMSD vs Score')
    #         plt.show()

    #         sns.scatterplot(x=real_rmsds, y=energies[1:])
    #         plt.title('RMSD vs Energy')
    #         plt.show()

    #         sns.scatterplot(x=preds[1:], y=energies[1:])
    #         plt.title('Pred vs Energy')
    #         plt.show()

In [10]:
params = [(i, smiles, cel.get_conf_ensemble(smiles)) for i, smiles in enumerate(test_smiles)]

In [13]:
with Pool(12) as pool :
    pool.map(dock_smiles, params)

Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/144/api_gold.conf
Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/0/api_gold.conf
Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/264/api_gold.conf
Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/192/api_gold.conf
Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/120/api_gold.conf
Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/72/api_gold.conf
Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/96/api_gold.conf
Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/48/api_gold.conf
Starting GOLD with conf file /home/be

RDKit ERROR: [16:54:21] non-ring atom 8 marked aromatic
RDKit ERROR: [16:54:21] non-ring atom 8 marked aromatic
[16:54:21] non-ring atom 8 marked aromatic
[16:54:21] non-ring atom 8 marked aromatic
RDKit ERROR: [16:54:21] non-ring atom 8 marked aromatic
RDKit ERROR: [16:54:21] non-ring atom 8 marked aromatic
RDKit ERROR: [16:54:21] non-ring atom 8 marked aromatic
RDKit ERROR: [16:54:21] non-ring atom 8 marked aromatic
[16:54:21] non-ring atom 8 marked aromatic
[16:54:21] non-ring atom 8 marked aromatic
[16:54:21] non-ring atom 8 marked aromatic
[16:54:21] non-ring atom 8 marked aromatic


Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/97/api_gold.conf
Setting up GOLD environment...
GOLD Version 2021.2.0
Running:
 
     "/home/benoit/CCDC/Discovery_2021/GOLD/gold/d_linux_64/bin/gold_linux_64" "/home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/97/api_gold.conf"

0
Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/1/api_gold.conf
Setting up GOLD environment...
GOLD Version 2021.2.0
Running:
 
     "/home/benoit/CCDC/Discovery_2021/GOLD/gold/d_linux_64/bin/gold_linux_64" "/home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/1/api_gold.conf"

0
Error with smiles O=C1C[N@@+]23Cc4cccc[n+]4[Fe+7]245(O1)OC(=O)C[N@+]4(Cc1cccc[n+]15)[C@@H]1CCCC[C@H]13
'NoneType' object has no attribute 'CalcEnergy'
Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/169/api_gold.conf
0
Setting up

RDKit WARNING: [16:54:24] unknown|48|mol2|1|dock1: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [16:54:24] unknown|48|mol2|1|dock53: warning - O.co2 with non C.2 or S.o2 neighbor.
[16:54:24] unknown|48|mol2|1|dock1: warning - O.co2 with non C.2 or S.o2 neighbor.
[16:54:24] unknown|48|mol2|1|dock53: warning - O.co2 with non C.2 or S.o2 neighbor.


0


...

RDKit WARNING: [19:54:03] unknown|1073|mol2|1|dock39: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [19:54:03] unknown|1073|mol2|1|dock36: warning - O.co2 with non C.2 or S.o2 neighbor.
[19:54:03] unknown|1073|mol2|1|dock39: warning - O.co2 with non C.2 or S.o2 neighbor.
[19:54:03] unknown|1073|mol2|1|dock36: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [19:54:03] unknown|1073|mol2|1|dock39: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [19:54:03] unknown|1073|mol2|1|dock36: warning - O.co2 with non C.2 or S.o2 neighbor.
[19:54:03] unknown|1073|mol2|1|dock39: warning - O.co2 with non C.2 or S.o2 neighbor.
[19:54:03] unknown|1073|mol2|1|dock36: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [19:54:03] unknown|1073|mol2|1|dock57: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [19:54:03] unknown|1073|mol2|1|dock57: warning - O.co2 with non C.2 or S.o2 neighbor.


Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/1074/api_gold.conf


[19:54:03] unknown|1073|mol2|1|dock57: warning - O.co2 with non C.2 or S.o2 neighbor.
[19:54:03] unknown|1073|mol2|1|dock57: warning - O.co2 with non C.2 or S.o2 neighbor.


Setting up GOLD environment...
GOLD Version 2021.2.0
Running:
 
     "/home/benoit/CCDC/Discovery_2021/GOLD/gold/d_linux_64/bin/gold_linux_64" "/home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/1074/api_gold.conf"

0
Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/1075/api_gold.conf
Setting up GOLD environment...
GOLD Version 2021.2.0
Running:
 
     "/home/benoit/CCDC/Discovery_2021/GOLD/gold/d_linux_64/bin/gold_linux_64" "/home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/1075/api_gold.conf"

0
Starting GOLD with conf file /home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/1076/api_gold.conf
Setting up GOLD environment...
GOLD Version 2021.2.0
Running:
 
     "/home/benoit/CCDC/Discovery_2021/GOLD/gold/d_linux_64/bin/gold_linux_64" "/home/benoit/bioactive_conformation_predictor/gold_docking_postprocessing/1076/api_gold.conf"

0
Starting GOLD with conf file /hom

RDKit WARNING: [19:55:06] unknown|1079|mol2|1|dock8: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [19:55:06] unknown|1079|mol2|1|dock29: warning - O.co2 with non C.2 or S.o2 neighbor.
[19:55:06] unknown|1079|mol2|1|dock8: warning - O.co2 with non C.2 or S.o2 neighbor.
[19:55:06] unknown|1079|mol2|1|dock29: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [19:55:07] unknown|1079|mol2|1|dock29: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [19:55:07] unknown|1079|mol2|1|dock14: warning - O.co2 with non C.2 or S.o2 neighbor.
[19:55:07] unknown|1079|mol2|1|dock29: warning - O.co2 with non C.2 or S.o2 neighbor.
[19:55:07] unknown|1079|mol2|1|dock14: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [19:55:07] unknown|1079|mol2|1|dock8: warning - O.co2 with non C.2 or S.o2 neighbor.
RDKit WARNING: [19:55:07] unknown|1079|mol2|1|dock29: warning - O.co2 with non C.2 or S.o2 neighbor.
[19:55:07] unknown|1079|mol2|1|dock8: warning - O.co2